In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.describe()

# Observations

1. **Age** is minimum at 40 which tells us that no one with the age below than 40 haas been recorded in this dataset. It maybe because no patients were ever admitted below 40 yrs of age.
2. **Creatinine Phosphokinase** is at a maximum of 7861. But, 75% quartile is at 582 which is low. That means that almost 75% of patients do not have very high CPK.

Similarly, other conclusions have been drawn from this.

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

### Good, there are no null values

In [ ]:
df.info()

In [ ]:
# Only 1 'age' column is in float so let's convert the whole column to an int.
df['age'] = df['age'].astype(int)

In [ ]:
df.info()

# Exploratory Data Analysis (EDA)

In [ ]:
px.histogram(x = 'age', y = 'DEATH_EVENT', data_frame = df, color = 'sex', 
             labels = {'age' : 'Age Group', 'DEATH_EVENT' : 'Number of Deaths'}, barmode='group')

In [ ]:
px.histogram(data_frame = df, x = 'creatinine_phosphokinase', y = 'DEATH_EVENT', color = 'sex', barmode = 'group', range_x = (0, 2500), 
            labels = {'creatinine_phosphokinase' : 'Amount of Creatinine Phosphokinase (Lower is Better)', 'Death_Event' : 'Number of Deaths'})

# Train Test Split

In [ ]:
X = df.drop(['DEATH_EVENT'], axis = 1)
y = df['DEATH_EVENT']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

# Feature Scaling

In [ ]:
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

# Dimensionality Reduction

In [ ]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components = 2)
#X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)

# Model Building

## XGBoost

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier(learning_rate = 0.5, gamma = 0.5)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5],
        'subsample': [1.0, 1.5, 2.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [10],
        'learning_rate': [0.5]
        }

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator = classifier, param_grid = params, n_jobs = -1, 
                    cv = 10, verbose = 4)

grid.fit(X_train,y_train)

In [ ]:
#{'colsample_bytree': 0.8,
#'gamma': 0.7,
# 'learning_rate': 0.3,
# 'max_depth': 6,
# 'min_child_weight': 5,
# 'subsample': 1.0}


#{'colsample_bytree': 0.8,
# 'gamma': 1,
# 'learning_rate': 0.3,
# 'max_depth': 4,
# 'min_child_weight': 5,
# 'subsample': 1.0}


grid.best_params_

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot = True)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_rf)

## Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

In [ ]:
svm_pred = svc.predict(X_test)

In [ ]:
accuracy_score(y_test, svm_pred)

------------------------------------------------------

# Conclusion: Both RandomForest and XGBoost gave us the same accuracy of 85 & 86.6% respectively even after being tuned a bit.